In [29]:
import torch
from torchvision import models
import torch.nn as nn
from getData import read_data

In [30]:
# Define MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(4, 10),  # 4 features to 10 hidden nodes
            nn.ReLU(),
            nn.Linear(10, 1),
            nn.Sigmoid()  # Output a single probability for binary classification
        )
    
    def forward(self, x):
        return self.layers(x)
    
# Load the models
mlp_model = MLP()
mlp_model.load_state_dict(torch.load('mlp_model.pth'))
mlp_model.eval()

image_model = models.resnet18(weights='ResNet18_Weights.DEFAULT')
num_ftrs = image_model.fc.in_features
image_model.fc = nn.Linear(num_ftrs, 2)
image_model.load_state_dict(torch.load('image_model.pth'))
image_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [36]:
image, stats = read_data()

Data has been written to CSV.
Image downloaded successfully and saved as c:\Users\jackm\Desktop\projects\fog_predictor\Images\Prediction\train\2024-05-23_11-46-04.jpg!


In [32]:
print(stats)

tensor([61.0000, 77.1700, 53.0000,  1.0000])


In [33]:
def predict_mlp(stats):
    # Read the most recent data entry
    data = stats
    
    # Make prediction
    with torch.no_grad():
        prediction = mlp_model(data)
    
    # Interpret the result
    probability = prediction.item()
    result = "Fog" if probability >= 0.5 else "No Fog"
    probability_percent = probability * 100  # Convert to percentage
    
    return {"probability": f"{probability_percent:.4f}%", "prediction": result}

In [34]:
predict_mlp(stats)

{'probability': '0.0006%', 'prediction': 'No Fog'}

In [35]:
def predict_image(image):
    image = image.unsqueeze(0)
    with torch.no_grad():
        output = image_model(image)
    
    # Get the predicted class probabilities
    probabilities = torch.softmax(output, dim=1)
    
    # Get the predicted class (0 or 1 for binary classification)
    predicted_class = probabilities.argmax(dim=1).item()
    
    # Get the probability of the predicted class
    probability = probabilities[0, predicted_class].item()
    result = "No Fog" if predicted_class == 1 else "Fog"
    probability_percent = probability * 100  # Convert to percentage
    
    
    return {"probability": f"{probability_percent:.4f}%", "prediction": result}

In [37]:
predict_image(image)

{'probability': '96.9228%', 'prediction': 'Fog'}